In [3]:
import sys
import pandas as pd
import os

def load_pickle(file_path):
    with open(file_path, 'rb') as handle:
        return pickle.load(handle)
        
def save_pickle(obj, filepath): 
    with open(filepath, 'wb') as handle:
        pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)

Для корректной работы KeplerGl в jupyter notebook: https://docs.kepler.gl/docs/keplergl-jupyter

In [2]:
os.makedirs('configs', exist_ok=True)

In [4]:
df_roads = pd.read_csv("data/train/roads_dataset.csv")
df_transport = pd.read_csv("data/train/routes_dataset.csv")
df_population = pd.read_csv("data/train/rosstat_population_all_cities.csv")
df_isochrones = pd.read_csv("data/train/isochrones_walk_dataset.csv")
df_stops = pd.read_csv("data/train/osm_stops.csv")
df_companies = pd.read_csv("data/train/osm_amenity.csv")

## Дороги
**highway** - тип дороги

**geometry** - shapely геометрия дороги

**oneway** - признак одностороннего движения

**name** - название улицы

**city** - город

In [30]:
df_roads.rename(columns={'osmid': 'id', 'geometry':'coord'}, inplace=True)
df_roads.head()

,id,highway,coord,oneway,name,city
0,33997807,primary_link,"LINESTRING (30.3314215 59.895094, 30.3311238 5...",False,NaN,Санкт-Петербург
1,201380605,primary,"LINESTRING (30.3314215 59.895094, 30.3318177 5...",True,Лиговский проспект,Санкт-Петербург
2,33997784,primary_link,"LINESTRING (30.3375246 59.8992885, 30.3379026 ...",False,NaN,Санкт-Петербург
3,178581714,primary,"LINESTRING (30.3375246 59.8992885, 30.3372282 ...",True,Лиговский проспект,Санкт-Петербург
4,4459876,primary,"LINESTRING (30.3395524 59.9017314, 30.3398299 ...",True,Воздухоплавательная улица,Санкт-Петербург


## Маршруты
**geometry** - shapely геометрия маршрута

**type** - тип общественного транспорта

**city** - город

**name** - идентификатор маршрута

In [21]:
df_transport.tail()

,geometry,type,city,name
1461,"LINESTRING (83.29533000000001 54.935855, 83.29...",shuttle_bus,Новосибирск,20cecfcf3ffe1eaf431ed018cfc50661
1462,"LINESTRING (83.108234 54.863827, 83.098676 54....",shuttle_bus,Новосибирск,44f683a84163b3523afe57c2e008bc8c
1463,"LINESTRING (83.615652 55.301539, 83.61573 55.3...",shuttle_bus,Новосибирск,9b8619251a19057cff70779273e95aa6
1464,"LINESTRING (82.99458199999999 55.135701, 82.99...",shuttle_bus,Новосибирск,730fb961e16877c20542d24b123d6ceb
1465,"LINESTRING (82.923587 55.016839, 82.9236559999...",shuttle_bus,Новосибирск,0e6641709663dd85809abae67c3f4358


## Население
**geo_h3_10** - https://eng.uber.com/h3/ : хексагона - полигон в виде соты

**city** - город

**population** - насление в хексагоне

**lat, lon** - широта/долгота центра хексагоны

In [6]:
df_population.head()

,geo_h3_10,city,population,lat,lon
0,8a0bb0200907fff,Новосибирск,12,54.942110,83.146356
1,8a0bb0200927fff,Новосибирск,8,54.942144,83.144157
2,8a0bb020092ffff,Новосибирск,28,54.943220,83.145270
3,8a0bb0200937fff,Новосибирск,12,54.941034,83.145243
4,8a0bb0200977fff,Новосибирск,10,54.944296,83.146384


## Изохроны 

https://medium.com/@urbica/%D0%B3%D0%B0%D0%BB%D1%8C%D1%82%D0%BE%D0%BD-c7360acf1ec4

**geo_h3_10** - https://eng.uber.com/h3/ : хексагона - полигон в виде соты

**city** - город

**walk_15min** - shapely геометрия 15 минутной пешей доступности из точки (lat, lon)

**lat, lon** - широта/долгота центра хексагоны

In [19]:
df_isochrones.tail()

,city,walk_15min,geo_h3_10,lat,lon
47092,Екатеринбург,"POLYGON ((60.595314 56.77692, 60.594258 56.776...",8a10dc936d97fff,56.767846,60.585314
47093,Екатеринбург,"POLYGON ((60.596524 56.777762, 60.594996 56.77...",8a10dc936d9ffff,56.768217,60.587524
47094,Екатеринбург,"POLYGON ((60.596222 56.777785, 60.595956 56.77...",8a10dc936da7fff,56.769914,60.584222
47095,Екатеринбург,"POLYGON ((60.597432 56.778491, 60.596432 56.77...",8a10dc936daffff,56.770285,60.586432
47096,Екатеринбург,"POLYGON ((60.586663 56.775702, 60.586524 56.77...",8a10dc936db7fff,56.768694,60.583663


### Магазины-кампании и прочее 
**geo_h3_10** - https://eng.uber.com/h3/ : хексагона - полигон в виде соты

**city** - город

**lat, lon** - широта/долгота центра хексагоны

**остальные колонки** - число организаций типа "название колонки" в хексагоне

In [27]:
df_companies.fillna(0).head()

,geo_h3_10,city,Автозапчасти для иномарок,Авторемонт и техобслуживание (СТО),Алкогольные напитки,Аптеки,Банки,Быстрое питание,Доставка готовых блюд,Женская одежда,...,Постаматы,Продуктовые магазины,Пункты выдачи интернет-заказов,Рестораны,Страхование,Супермаркеты,Цветы,Шиномонтаж,lat,lon
0,8a0bb0200a5ffff,Новосибирск,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,54.953869,83.163008
1,8a0bb0200b37fff,Новосибирск,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,54.948563,83.153038
2,8a0bb0201cd7fff,Новосибирск,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,54.958064,83.174064
3,8a0bb0201d97fff,Новосибирск,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,54.953799,83.167407
4,8a0bb0204027fff,Новосибирск,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54.935864,83.126481


## Остановки
**osm_id** - идентификатор 

**geo_h3_10** - https://eng.uber.com/h3/ : хексагона - полигон в виде соты

**lat, lon** - широта/долгота центра хексагоны

**type** - тип остановки

**city** - город

**name** - название остановки

In [22]:
df_stops.tail()

,osm_id,name,lat,lon,type,city,geo_h3_10
12311,9732849975,улица Симонова,60.063763,30.327582,bus_stop,Санкт-Петербург,8a1106a81107fff
12312,9732849977,улица Композиторов,60.064654,30.321466,bus_stop,Санкт-Петербург,8a1106a818a7fff
12313,9732849979,улица Жени Егоровой,60.065848,30.315515,bus_stop,Санкт-Петербург,8a1106a81907fff
12314,9756417575,Проспект Художников,60.043024,30.350641,bus_stop,Санкт-Петербург,8a1106a83187fff
12315,9756417576,Проспект Художников,60.043509,30.349097,bus_stop,Санкт-Петербург,8a1106a83187fff


### Визуализируем данные

In [12]:
map_config_path = "configs/map-config.pkl"
if os.path.isfile(map_config_path):
    config = load_pickle(map_config_path)
else:
    config = {}

In [13]:
map_kepler = KeplerGl(height = 750, config=config)
map_kepler.add_data(df_roads, 'дороги')
map_kepler.add_data(df_transport, 'маршруты транспорта')
map_kepler.add_data(df_population.drop('geo_h3_10', axis = 1), 'Население')
map_kepler.add_data(df_isochrones.drop('geo_h3_10', axis = 1), 'Пешая доступность')
map_kepler.add_data(df_stops.drop('geo_h3_10', axis = 1), 'Остановки')
map_kepler

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['Население'], 'id': 'x6eim9x…

In [28]:
save_pickle(map_kepler.config, "configs/map-config.pkl")